In [1]:
%matplotlib qt

import neuroseries as nts

import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
import scipy.ndimage
import os

import scipy.signal
import scipy.stats

import cv2
import datetime

import bk.load 
import bk.plot
import bk.compute

import sklearn.manifold as skm
import scipy.stats
from matplotlib import cm

In [2]:
def pourcent_freezing(interval, freezing_intervals): 
    freezing_intervals_in_interval = freezing_intervals.intersect(interval)
    total_time_freezing = np.sum(freezing_intervals_in_interval.end-freezing_intervals_in_interval.start)
    total_time = interval.end-interval.start
    pourcentage = (total_time_freezing/total_time)*100
    return pourcentage

def all_pourcentages(intervals, freezing_intervals):
    pourcentages = []
    for i in range(len(intervals)):
        interval = intervals.loc[i]
        pourcentages.append(pourcent_freezing(interval, freezing_intervals))
    return pourcentages

def GetStartExp(Exp,Fs = 20000):
    
    if isinstance(Exp[0],(np.bool_,bool)):
        Exp = list(map(int,Exp))
    
    diff_Exp = np.diff(Exp)
    
    t_start = np.where(diff_Exp == 1)[0]    
    return t_start/Fs

## Analysis for classical fear conditionning 

In [5]:
context_sess = ['Y:\classical_fear_conditionning\Rat09-20201118\Rat09_201118_180813',
                'Y:\classical_fear_conditionning\Rat10-20201118\Rat10_201118_182234', 
                'Y:\classical_fear_conditionning\Rat11-20201117\Rat11_201117_172705',
                'Y:\classical_fear_conditionning\Rat12-20201117\Rat12_201117_171302', 
                'Y:\classical_fear_conditionning\Rat01-20200630\Rat01_200630_183149',
                'Y:\classical_fear_conditionning\Rat02-20200707\Rat02_200707_172929']
fear_sess = ['Y:\classical_fear_conditionning\Rat09-20201118\Rat09_201118_110003',
             'Y:\classical_fear_conditionning\Rat10-20201118\Rat10_201118_111531',
             'Y:\classical_fear_conditionning\Rat11-20201117\Rat11_201117_112130',
             'Y:\classical_fear_conditionning\Rat12-20201117\Rat12_201117_110146',
            'Y:\classical_fear_conditionning\Rat01-20200630\Rat01_200630_114732',
            'Y:\classical_fear_conditionning\Rat02-20200707\Rat02_200707_114611']

tone_sess = ['Y:\classical_fear_conditionning\Rat09-20201118\Rat09_201118_170529', 
        'Y:\classical_fear_conditionning\Rat10-20201118\Rat10_201118_172028', 
        'Y:\classical_fear_conditionning\Rat11-20201117\Rat11_201117_183012',
        'Y:\classical_fear_conditionning\Rat12-20201117\Rat12_201117_181452', 
        'Y:\classical_fear_conditionning\Rat01-20200630\Rat01_200630_171900', 
        'Y:\classical_fear_conditionning\Rat02-20200707\Rat02_200707_183427' ]


intervals = np.linspace(0,570,20)
starts = intervals[0:-1]
ends = intervals[1:]
intervals = nts.IntervalSet(starts,ends,time_units='s')

digitalin_tone = [3,3,3,3,2,2]
digitalin_shock = [2,2,2,2,1,1]

In [6]:
t_start_exp_tone = [49.9391, 12.74455, 30.36235, 29.62595, 58.95, 17.8116]

ptotal = []
plt.figure()

for fichier,t_s,d_t in zip(tone_sess,t_start_exp_tone,digitalin_tone):
    os.chdir(fichier)
    pos = bk.load.DLC_pos()
    pos.index = pos.index
    tone = bk.compute.tone_intervals(bk.load.digitalin('digitalin.dat')[d_t,:])- t_s * 1_000_000
    speeds = bk.compute.speed(pos,2, columns_to_drop=['e_tail'])
    freeze = bk.compute.freezing_intervals(speeds, 25, mode='multiple_speed', clean=True) - t_s *1_000_000
    pourcents = all_pourcentages(intervals, freeze)
    ptotal.append(pourcents)
#     bk.plot.intervals(freeze,col = 'b')
    plt.plot(np.mean([starts,ends],0), pourcents, '-o',alpha = 0.5)
    
bk.plot.intervals(tone,'orange', alpha=0.5)    
plt.plot(np.mean([starts,ends],0),np.mean(ptotal,0),'*-r')
plt.legend(['Rat09', 'Rat10', 'Rat11', 'Rat12', 'Rat01', 'Rat02','Mean','Tone'])

Z:\Python\Maelle\Amy-Hpc-sleep-dynamics-python\bk\compute.py:161: RuntimeWarning: overflow encountered in square
  v = np.sqrt(x_speed**2 + y_speed**2)


In [60]:
t_start_exp_fear = [37.7209,29.46495,34.7118,25.34385,45.9524,55.00735]

ptotal = []
plt.figure()

for fichier,t_s,d_s in zip(fear_sess,t_start_exp_fear, digitalin_shock):
    os.chdir(fichier)
    pos = bk.load.DLC_pos()
    pos.index = pos.index
    tone = bk.compute.tone_intervals(bk.load.digitalin('digitalin.dat')[d_s+1,:])- t_s * 1_000_000
    shock = bk.compute.tone_intervals(bk.load.digitalin('digitalin.dat')[d_s,:])- t_s * 1_000_000
    speeds = bk.compute.speed(pos,2, columns_to_drop=['e_tail'])
    freeze = bk.compute.freezing_intervals(speeds, 25, mode='multiple_speed', clean=True) - t_s *1_000_000
    pourcents = all_pourcentages(intervals, freeze)
    ptotal.append(pourcents)
#     bk.plot.intervals(freeze,col = 'b')
    plt.plot(np.mean([starts,ends],0), pourcents, '-o',alpha = 0.5)
bk.plot.intervals(shock,'green',alpha = 0.5)    
bk.plot.intervals(tone,'orange', alpha=0.5)    
plt.plot(np.mean([starts,ends],0),np.mean(ptotal,0),'*-r')
plt.legend(['Rat09', 'Rat10', 'Rat11', 'Rat12', 'Rat01', 'Rat02','Mean','Shock','Tone'])

In [62]:
t_start_exp_context = [34,25,36,26,38,58]

ptotal = []
plt.figure()

for fichier,t_s in zip(context_sess,t_start_exp_context):
    os.chdir(fichier)
    pos = bk.load.DLC_pos()
    pos.index = pos.index
    speeds = bk.compute.speed(pos,2, columns_to_drop=['e_tail'])
    freeze = bk.compute.freezing_intervals(speeds, 25, mode='multiple_speed', clean=True) - t_s *1_000_000
    pourcents = all_pourcentages(intervals, freeze)
    ptotal.append(pourcents)
#     bk.plot.intervals(freeze,col = 'b')
    plt.plot(np.mean([starts,ends],0), pourcents, '-o',alpha = 0.5)
plt.plot(np.mean([starts,ends],0),np.mean(ptotal,0),'*-r')
plt.legend(['Rat09', 'Rat10', 'Rat11', 'Rat12', 'Rat01', 'Rat02','Mean'])

In [61]:
for fichier in context_sess:
    os.chdir(fichier)
    pos = bk.load.DLC_pos()
    print(pos.as_units('s').index[-1]-575)
#     f_tone = GetStartExp(bk.load.digitalin('digitalin.dat')[1,:])
#     print(f_tone[f_tone>0])
    
    


34.458774999999946
[  3.0225    3.1475    3.2725  203.08695 353.30415 503.51535]
25.68372499999998
[  3.0225    3.1475    3.2725  203.08695 353.30415 503.51535]
36.234050000000025
[  3.0225    3.1475    3.2725  203.08695 353.30415 503.51535]
26.03037500000005
[  3.0225    3.1475    3.2725  203.08695 353.30415 503.51535]
38.39945
[  3.0225    3.1475    3.2725  203.08695 353.30415 503.51535]
58.11689999999999
[  3.0225    3.1475    3.2725  203.08695 353.30415 503.51535]


## Analysis for PTSD fear conditionning 

In [3]:
neutral_sess = ['Y://fear_conditionning_ptsd\Rat13\Rat13-20201202\Rat13_201202_183404',
               'Y://fear_conditionning_ptsd\Rat14\Rat14-20201203\Rat14_201203_165203',
               'Y://fear_conditionning_ptsd\Rat15\Rat15-20201130\Rat15_201130_164002',
               'Y://fear_conditionning_ptsd\Rat16\Rat16-20201201\Rat16_201201_172742']



context_sess = ['Y://fear_conditionning_ptsd\Rat13\Rat13-20201203\Rat13_201203_154214',
               'Y://fear_conditionning_ptsd\Rat14\Rat14-20201204\Rat14_201204_172709',
               'Y://fear_conditionning_ptsd\Rat15\Rat15-20201201\Rat15_201201_155055',
               'Y://fear_conditionning_ptsd\Rat16\Rat16-20201202\Rat16_201202_171552']

fear_sess = ['Y://fear_conditionning_ptsd\Rat13\Rat13-20201203\Rat13_201203_112541',
            'Y://fear_conditionning_ptsd\Rat14\Rat14-20201204\Rat14_201204_113458',
            'Y://fear_conditionning_ptsd\Rat15\Rat15-20201201\Rat15_201201_115713',
            'Y://fear_conditionning_ptsd\Rat16\Rat16-20201202\Rat16_201202_132308']

tone_sess = ['Y://fear_conditionning_ptsd\Rat13\Rat13-20201203\Rat13_201203_164205',
            'Y://fear_conditionning_ptsd\Rat14\Rat14-20201204\Rat14_201204_161859',
            'Y://fear_conditionning_ptsd\Rat16\Rat16-20201202\Rat16_201202_182329']


intervals = np.linspace(0,240,13)
starts = intervals[0:-1]
ends = intervals[1:]
intervals = nts.IntervalSet(starts,ends,time_units='s')

digitalin_tone = [3,3,3,3]
digitalin_shock = [2,2,2,2]

In [4]:
t_start_exp_neutral = [36.98515,25.79435,36.52745,34.03365]

ptotal = []
plt.figure()

for fichier,t_s in zip(neutral_sess,t_start_exp_neutral):
    os.chdir(fichier)
    pos = bk.load.DLC_pos()
    pos.index = pos.index
    speeds = bk.compute.speed(pos,2, columns_to_drop=['e_tail'])
    freeze = bk.compute.freezing_intervals(speeds, 25, mode='multiple_speed', clean=True) - t_s *1_000_000
    pourcents = all_pourcentages(intervals, freeze)
    ptotal.append(pourcents)
#     bk.plot.intervals(freeze,col = 'b')
    plt.plot(np.mean([starts,ends],0), pourcents, '-o',alpha = 0.5)
plt.plot(np.mean([starts,ends],0),np.mean(ptotal,0),'*-r')
plt.legend(['Rat13', 'Rat14','Rat15','Rat16','Mean'])
plt.title('Pourcent freezing in neutral context before fear conditionning')

Y://fear_conditionning_ptsd\Rat13\Rat13-20201202\Rat13_201202_183404
Y://fear_conditionning_ptsd\Rat14\Rat14-20201203\Rat14_201203_165203
Y://fear_conditionning_ptsd\Rat15\Rat15-20201130\Rat15_201130_164002


ValueError: Length mismatch: Expected axis has 8194 elements, new values have 8195 elements

In [13]:
t_start_exp_fear = [43.6102,61.40995,75.8984,36.58545]

ptotal = []
plt.subplot(3,1,1)

for fichier,t_s,d_s in zip(fear_sess,t_start_exp_fear, digitalin_shock):
    os.chdir(fichier)
    pos = bk.load.DLC_pos()
    pos.index = pos.index
    tone = bk.compute.tone_intervals(bk.load.digitalin('digitalin.dat')[d_s+1,:])- t_s * 1_000_000
    shock = bk.compute.tone_intervals(bk.load.digitalin('digitalin.dat')[d_s,:])- t_s * 1_000_000
    speeds = bk.compute.speed(pos,2, columns_to_drop=['e_tail'])
    freeze = bk.compute.freezing_intervals(speeds, 25, mode='multiple_speed', clean=True) - t_s *1_000_000
    pourcents = all_pourcentages(intervals, freeze)
    ptotal.append(pourcents)
#     bk.plot.intervals(freeze,col = 'b')
    plt.plot(np.mean([starts,ends],0), pourcents, '-o',alpha = 0.3)
bk.plot.intervals(shock,'green',alpha = 0.5)    
bk.plot.intervals(tone,'orange', alpha=0.5)    
plt.plot(np.mean([starts,ends],0),np.mean(ptotal,0),'*-',color = 'black',linewidth=2)
plt.legend(['Rat13', 'Rat14', 'Rat15', 'Rat16','Mean','Shock','Tone'])
plt.title('Pourcent freezing during fear conditionning')

Text(0.5, 1.0, 'Pourcent freezing during fear conditionning')

In [14]:
t_start_exp_tone = [31.5871,32.76115,5.97]

ptotal = []
plt.subplot(3,1,2)

for fichier,t_s,d_t in zip(tone_sess,t_start_exp_tone,digitalin_tone):
    os.chdir(fichier)
    pos = bk.load.DLC_pos()
    pos.index = pos.index
    tone = bk.compute.tone_intervals(bk.load.digitalin('digitalin.dat')[d_t,:])- t_s * 1_000_000
    speeds = bk.compute.speed(pos,2, columns_to_drop=['e_tail'])
    freeze = bk.compute.freezing_intervals(speeds, 25, mode='multiple_speed', clean=True) - t_s *1_000_000
    pourcents = all_pourcentages(intervals, freeze)
    ptotal.append(pourcents)
#     bk.plot.intervals(freeze,col = 'b')
    plt.plot(np.mean([starts,ends],0), pourcents, '-o',alpha = 0.3)
    
bk.plot.intervals(tone,'orange', alpha=0.5)    
plt.plot(np.mean([starts,ends],0),np.mean(ptotal,0),'*-',color = 'black',linewidth=2)
plt.legend(['Rat13', 'Rat14', 'Rat16','Mean','Tone'])
plt.title('Pourcent freezing in tone retrieval')

Text(0.5, 1.0, 'Pourcent freezing in tone retrieval')

In [15]:
t_start_exp_context = [23.49815,31.56275,48.84175,24.49015]

ptotal = []
plt.subplot(3,1,3)

for fichier,t_s in zip(context_sess,t_start_exp_context):
    os.chdir(fichier)
    pos = bk.load.DLC_pos()
    pos.index = pos.index
    speeds = bk.compute.speed(pos,2, columns_to_drop=['e_tail'])
    freeze = bk.compute.freezing_intervals(speeds, 25, mode='multiple_speed', clean=True) - t_s *1_000_000
    pourcents = all_pourcentages(intervals, freeze)
    ptotal.append(pourcents)
#     bk.plot.intervals(freeze,col = 'b')
    plt.plot(np.mean([starts,ends],0), pourcents, '-o',alpha = 0.3)
plt.plot(np.mean([starts,ends],0),np.mean(ptotal,0),'*-',color = 'black',linewidth=2)
plt.legend(['Rat13', 'Rat14', 'Rat15', 'Rat16','Mean'])
plt.title('Pourcent freezing in context retrieval')

Text(0.5, 1.0, 'Pourcent freezing in context retrieval')

In [15]:
for subsession in neutral_sess:
    os.chdir(subsession)
    f_tone = GetStartExp(bk.load.digitalin('digitalin.dat')[1,:])
    print(f_tone[f_tone>0])

[36.98515]
[25.79435]
[36.52745]
[34.03365]
